In [2]:
# https://huggingface.co/docs/transformers/training
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [1]:
from datasets import load_dataset

ds=load_dataset('.',data_files={'train': ['../rajoy_train.jsonl'],
                             'test':['../rajoy_test.jsonl']})

Using custom data configuration .-b1c978082bff5d3c
Reusing dataset json (/home/azureuser/.cache/huggingface/datasets/json/.-b1c978082bff5d3c/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)

ds=ds.map(tokenize_function,batched=True)

Parameter 'function'=<function tokenize_function at 0x7f3938ea0040> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/29 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['index', 'Text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 28962
    })
    test: Dataset({
        features: ['index', 'Text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 23697
    })
})

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [6]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds.train,
    eval_dataset=ds.test,
    compute_metrics=compute_metrics,
)